# Robotics Foundations (H) - Lab 1

## ILOs
In this lab, you will get to learn and practice how to create a ROS package, understand the file system and compilation tools, and the simulation framework of Gazebo. At the end of the lab, you should be able to:

* Use the simulation facilities in ROS
* Run a ROS node.
* Move a simulated robot, named Baxter, using the keyboard

![baxter_glasgow.png](imgs/baxter_glasgow.png)
<div style="text-align:center"><b>Figure 1: The Baxter robot at the School of Computing Science</b><br></div>

## ROS in a nutshell!

**NOTE:** If you have attended Lecture 2 "Introduction to ROS", you do not need to read this section! Jump to the "Creating a ROS package" section :)

The main aim of ROS is **to provide OS-like functionalities for specialised robotic agents, under an open-source license**. The creators based ROS around five fundamental design principles: peer-to-peer, tools-based, multi-lingual, thin, and open-source. It currently supports four languages: C++, Python, Octave and LISP with other language ports in development. To be language-independent, ROS uses a language-neutral interface to define messages sent between modules. So, the services it offers are similar to the ones found in most software middleware (e.g. XMLRPC servers and Django in WAD2!). For instance, ROS offers:

- Hardware abstraction, which enables the development of portable code which can interface with a large number of robot platforms
- Low-level device control, facilitating robotic control
- Inter-process communication via message passing
- Software package management, which ensures the framework is easily extensible

At the core of all the programs running on ROS is an anonymised form of the publisher-subscriber pattern. Processes are termed **nodes** in ROS. Nodes form part of a ROS graph (Figure 1) that keeps track of running nodes and all the communications between them at a fine-grained scale. Since most robotic systems are assumed to comprise numerous components, all requiring control or I/O services, the ROS node architecture expects that each computation is delegated to a separate node, thus reducing the complexity involved in developing monolithic control systems. 

![ros_graph.png](imgs/ros_graph.png)
<div style="text-align:center"><b>Figure 1: Example of a ROS node graph</b><br></div>

A benefit of using ROS is that it allows the debugging of a node alongside other running nodes. The latter is facilitated by the modular structure. It also means people working in robotics do not have to reinvent the wheel for each project, since ROS nodes and packages can be easily shared, creating a vibrant community. Additionally, all other nodes can be started and left running, and developers can edit and restart the ROS node they are working on. The ability to stop, edit and restart ROS codes are incredibly beneficial and handy as the software system can get very complicated! ([http://www.willowgarage.com/sites/default/files/icraoss09-ROS.pdf](http://www.willowgarage.com/sites/default/files/icraoss09-ROS.pdf)).

Newly-created nodes can communicate using:

- Topics
- RPC Services
- Parameter Server

The most common communication mechanism involves transmitting and receiving messages asynchronously on a strongly-typed named bus or log called a **topic**. Nodes can create, publish or subscribe to a topic, and send and receive data in the form of messages of a predefined type, at a set frequency measured in Hz. Built for unidirectional streaming communication topics, the software developer (aka you!) benefits from anonymous publish/subscribe semantics, thus removing communicating nodes' awareness of each other.

However, in cases where the many-to-many transport provided by topics is not appropriate, ROS offers a form of node interaction based on the request/reply paradigm of remote procedure calls. These **services** are defined by a pair of messages, of which one is the request sent to a node, and the other is the reply.

The **parameter server** takes the form of a globally-viewable dictionary of typically static, non-binary data points. The server is designed primarily for the storage of shared configuration parameters, and does not uphold any premise of high performance and should not be seen as a global variable store.

The above three communication types support the smooth integration of custom user code within a ROS ecosystem and unify the different APIs that would typically be needed to access relevant system information and low-level drivers of a robot (e.g. sensor data, actuator positions, motor positions, etc.).

## ROS Filesystem
Similar to an operating system, a ROS workspace is divided into folders, and these folders have files that describe their functionalities:

* **Packages**: Packages form the atomic level of ROS. A package has the minimum structure and content to create a program within ROS. It may have ROS runtime processes (nodes), configuration files, and so on. In the following sections, you will create a ROS package.
* **Package manifests**: Package manifests provide information about a package, licenses, dependencies, compilation flags, and so on. A package manifest is managed with a file called package.xml.
* **Message (msg) types**: A message is an information that a process sends to other processes. ROS has several standard types of messages. Message descriptions are stored in `<package name>/msg/MyMessageType.msg`.
* **Service (srv) types**: Service descriptions, stored in `<package name>/srv/MyServiceType.srv`, define the request and response data structures for services provided by each process in ROS.

## Creating a ROS package
To develop software in ROS for robots, you usually create a workspace where you will store ROS packages and related configurations and files. In general terms, the workspace is a folder tree that contains packages, those packages include your source files, and the workspace provides you with a way to compile those packages.

In this Virtual Machine, we have configured a ROS workspace for you with the [Baxter robot](https://en.wikipedia.org/wiki/Baxter_(robot)) SDK. To separate workspaces and allow you to save your progress, you need to create a new workspace which will be located in: `~/workspace/rf_ws` (`~` is a shortcut to the user's home directory). So, open a terminal and type the following (to open a terminal in the virtual machine, minimise this Firefox and you will see a shortcut to the default terminal in the Desktop):

``` bash
mkdir -p ~/rf_ws/src
cd ~/rf_ws/src
catkin_init_workspace
```

With the above commands, you would have created an `src` folder where your ROS packages will be located. Now, you need to create a package in the workspace called `lab1_pkg` that uses `rospy` (the Python ROS API) and some standard ROS message packages, robot messages and Baxter's SDK interface. For this, type the following commands in the terminal you opened recently (if you close it, you need to open and a new terminal and navigate to `~/rf_ws/src`):

``` bash
catkin_create_pkg lab1_pkg rospy geometry_msgs sensor_msgs control_msgs trajectory_msgs baxter_core_msgs baxter_interface
```
The first argument, `lab1_pkg`, is the name of your new package. The following arguments are the package names the new package depends on. You must include these because the ROS build system needs to know the package dependencies to keep the builds up-to-date efficiently and to generate any required dependencies. **Tip:** you can find out more about these ROS message packages and other packages by searching for them in Google, e.g. https://www.google.co.uk/search?q=geometry_msgs+ros

After running the `catkin_create_pkg` command, there will be a package directory called `lab1_pkg` inside the workspace. The Figure below shows the package in the file manager; to open the file manager, minimise Firefox or change virtual desktop by pressing `Ctrl+Alt+Left`, click on the "Home" icon, then navigate to `~/rf_ws/src`. If you double-click on the "lab1_pkg" folder, you will find the following files:

* `CMakeLists.txt`, a starting point for the build script for this package
* `package.xml`, machine-readable description of the package, including details such as its name, description, author, license, and which other packages it depends on to build and run.

You can right-click on either `CMakeLists.txt` or `package.xml` and open them to inspect the contents of both files. We have installed `Visual Studio Code` for you, and you can access it from the desktop or by right-clicking on a file.

![filestructure.png](imgs/filestructure.png)
<div style="text-align:center"><b>Figure 2: ROS package structure</b><br></div>

## Creating a ROS node
You can now create a minimal ROS node inside of the `lab1_pkg` package. The purpose of this node will be to communicate with a simulated robot (more in the following sections) and to drive the robot using the keyboard. The following code maps keyboard inputs into motion in the robot. To understand the code, click on the following link to open a Python ROS node.

<a href="../position_keyboard.py" target="_blank">Open position_keyboard.py</a>

A lot of the code is just setting up the system and its data structures. The most critical piece of code is `set_j` function, line 79, which commands the robot to move a joint. The other key block of code is within lines 124 and 142, which provides the UI, map keyboard strokes and calls `set_j` using lambda functions.
    
Let's now get it into ROS. For this, you need to create a Python script inside `lab1_pkg/src` and copy and paste the above code. Alternatively, press `Ctrl+Enter` or `Shift+Enter` to evaluate the following cell which will copy the file you inspected into `lab1_pkg/src`:

In [1]:
%%bash
cp ~/Desktop/RFLabs/lab1/position_keyboard.py ~/rf_ws/src/lab1_pkg/src/position_keyboard.py

Like all Python scripts in Linux, it is convenient to make it executable so ROS can run the node (select the following cell and press `Ctrl+Enter`):

In [2]:
%%bash
chmod +x ~/rf_ws/src/lab1_pkg/src/position_keyboard.py

Before running this node, you now need to "compile" your package to register it within the ROS `PATH`. For this, go to the terminal (or open a new terminal if you closed it) and type the following:

``` bash
cd ~/rf_ws
catkin_make
source devel/setup.bash
```

`catkin_make` invokes a script to "compile" (or in Unix jargon "make") all packages inside `src` in your workspace. As you know, Python scripts don't need to be compiled; however, `catkin_make` is required to create a valid ROS package. `source devel/setup.bash` adds all compiled ROS packages within `rf_ws/src` to the ROS PATH, so ROS knows where to look for your packages and nodes when you run them! You need to source (i.e. run `source devel/setup.bash`) your workspace for each terminal you open.

Now, you are ready to instantiate a Baxter in the simulator. You’ll use a simple simulated world to start with, by typing the following in a new terminal window (remember that you can hit the Tab key when typing in a terminal for autocompletion):

**NOTE:** you can open multiple terminals by pressing `Ctrl+T` on the terminal.

``` bash
roslaunch baxter_gazebo baxter_world.launch
```

**NOTE:** Don't close the "BaxterIO" window; otherwise the simulation will crash. This window simulates some physical buttons in Baxter, which are used to interact with the robot. We will not cover this in Robotic Foundations though.

**Note:** If you see that the simulation does not load Baxter into it, just press Ctrl+C in the terminal where you run the above command; then run the command again. Sometimes the simulation does not load correctly and it is a bug that has been reported for years! Usually, the second try will work just fine...

One of the annoying things while working with ROS is the need to open multiple terminals for each node. ROS creators have somewhat mitigated this by grouping nodes in `launch` files such as the one you just used to run the simulation above (rant over...). That is, [`roslaunch`](http://wiki.ros.org/roslaunch) is a tool that allows you to run several ROS nodes, declare ROS parameters, pass command line arguments to ROS nodes, and so on. A `roslaunch` file is structured as [XML](http://wiki.ros.org/roslaunch/XML) and run a [`roscore`](http://wiki.ros.org/roscore) in the background. The `roscore` is the central hub where topics, services and parameters are stored and must be running so ROS nodes can communicate with each other.

After the simulation finishes loading (a good clue is to look for a `Robot is disabled` message in the terminal), you can start the newly created ROS node in a different terminal as below. If `rosrun` cannot find your `lab1_pkg` package, run `source devel/setup.bash` while you are in your workspace, i.e. `~/rf_ws`.

``` bash
rosrun baxter_tools enable_robot.py -e
rosrun lab1_pkg position_keyboard.py
```

`enable_robot.py` is a ROS node from Baxter SDK that activates Baxter to receive messages and service calls. When `position_keyboard.py` is running, you can now press keys (press `?` for a full list of possible commands) to move Baxter's joints. Baxter has 7 joints on each arm plus a gripper. We encourage you to move all joints to explore Baxter's working volume. Bring the Gazebo window to the front, then the terminal running `position_keyboard.py` so you can observe what happens when you move Baxter's joint.

## Visualising Baxter
Within ROS, there exists a Robot Visualization (RViz) tool that lets you inspect the parameters, configurations and even examine what the robot sees and plans to do (more about this in coming labs). To start type in a new terminal (again, remember to source the new terminal!):

``` bash
rosrun rviz rviz
```

Then, select the `Add` button under the Displays sidebar. In the new window that pops up, scroll to `Robot Model`, and then hit ‘OK’ to add a visual representation of the robot’s current pose. At this point, you will get a big white lump as the robot. To fix this, you now need to set the `Fixed Frame` under `Global Options` to `base`; otherwise. In `RobotModel` on the left pane in RViz, expand the tree, and you should see the default parameters; the important parameter here is *Robot Description* which should point to the `robot_description` variable. `robot_description` is the name of the ROS parameter where Baxter's foward kinematics description, 3D meshes and mechanical properties are stored on the parameter server - you can query the parameter server in the terminal with:

``` bash
rosparam get /robot_description
```

and you'll get its contents -- more about this in lab2, skip for the moment. If everything is fine, you will see the following window with Baxter in it.

![rviz.png](imgs/rviz_baxter.png)
<div style="text-align:center"><b>Figure 2: RViz</b><br></div>

**In RViz and Gazebo, you can zoom, pan and rotate the camera view and angle. If you are using a laptop, you can do the same but if you want to rotate you have to press `SHIFT+left mouse button`.**

If you actuate Baxter using `position_keyboard.py`, you will notice that RViz replicates what you see in Gazebo. Gazebo simulates a Baxter robot, while RViz displays the internal state of the robot. This means that RViz will not work if you do not have either a simulated robot or a real robot!. When you get bored moving Baxter, press Ctrl-C to `position_keyboard.py` node as well as the Baxter simulation to stop your node and the simulation, respectively.

## Final thoughts

In this lab, you use the simulation facilities in ROS of which you will rely heavily on during the semester. You also run a ROS node and moved Baxter using the keyboard. 

In the following weeks, you will understand the inner workings of ROS and add extra capabilities to Baxter.


## References
* Quigley, M., Gerkey, B. and Smart, W.D., 2015. Programming Robots with ROS: a practical introduction to the Robot Operating System. O'Reilly Media, Inc.